# Path Finding

This notebook contains the scripts used to construct a directed weighted graph from the preprocessed data generated by notebook 01 and calculate the total number of hours required to travel between any 2 cities (in our case, we use London). The calculations are done using Dijkstra's algorithm

In [14]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.neighbors import BallTree
import networkx as nx
import math
import json
import matplotlib.pyplot as plt
import pickle
import sys
sys.path.append(str(Path(Path.cwd()).parent.resolve()))

from src.utils import is_east_of, travel_time

In [ ]:
DATA_PROC = Path("../data/processed/worldcities_processed.csv")
df = pd.read_csv(DATA_PROC)
df.head()

## Finding nearest cities

In [4]:
coords = np.radians(df[['lat', 'lon']].values)
tree = BallTree(coords, metric="haversine")

# query 4 because first neighbor is the point itself
distances, indices = tree.query(coords, k=4)
neighbor_ids = indices[:, 1:4]

In [5]:
neighbor_ids

array([[ 873342,  873599,  877115],
       [ 361002,  270635,  297445],
       [ 770972,  770971,  770984],
       ...,
       [1683844, 1683616, 1683389],
       [1683903, 1683283, 1683294],
       [1683925, 1683889, 1683446]], shape=(1683979, 3))

## Build a Directed Graph and save it for later

In [15]:
G = nx.DiGraph()

for i, row in df.reset_index().iterrows():
    node_id = int(row['index']) if 'index' in df.columns else i
    G.add_node(node_id, name=row['city'], country=row['country'], lat=row['lat'], lon=row['lon'])

for i in range(len(df)):
    a_country = df.iloc[i]['country']
    a_city = df.iloc[i]['city']
    a_lat = df.iloc[i]['lat']
    a_lon = df.iloc[i]['lon']
    for rank, neighbor_idx in enumerate(neighbor_ids[i], start=1):
        b = df.iloc[neighbor_idx]
        if is_east_of(a_lon, b['lon']):
            weight = travel_time(rank, a_country, b['country'], b['population'])
            G.add_edge(i, int(neighbor_idx), weight=weight)

In [16]:
GRAPH_PATH = Path("../artifacts/cities_graph.pkl")
GRAPH_PATH.parent.mkdir(exist_ok=True)
with open(GRAPH_PATH, "wb") as f:
    pickle.dump(G, f)

print("Graph saved:", GRAPH_PATH)

Graph saved: ..\artifacts\cities_graph.pkl
